In [1]:
import pandas as pd
from epiweeks import Week
from bulletin.data.sivep import sivep

pd.set_option('display.max_columns', None)

In [2]:
sivep = sivep()
sivep.read_all_database_files()

In [14]:
sivep_df = sivep.get_df()

In [15]:
sivep_df = sivep_df.loc[sivep_df['SG_UF'] == 'PR']
sivep_df = sivep_df.loc[sivep_df['SG_UF_NOT'] == 'PR']
sivep_df = sivep_df.loc[sivep_df['AMOSTRA'] == '1']
sivep_df.loc[sivep_df['CLASSI_FIN'].isnull(), 'CLASSI_FIN'] = '0'

In [16]:
sivep_df['DT_NOTIFIC'] = sivep_df['DT_NOTIFIC'].apply(lambda x: pd.to_datetime(x, format='%d/%m/%Y'))
sivep_df['SE'] = sivep_df['DT_NOTIFIC'].apply(lambda x: Week.fromdate(x))

In [17]:
tabela = sivep_df.groupby(['SE','CLASSI_FIN'])[['NU_NOTIFIC']].count().unstack()

In [18]:
tabela = tabela.droplevel(0, axis=1)

In [19]:
# tabela['Semana Epidemiológica'] = [ x.week for x in tabela.index.values]
# tabela = tabela.set_index('Semana Epidemiológica')

In [20]:
tabela['1'] = tabela['1'].add(tabela['2'])
tabela['1'] = tabela['1'].add(tabela['3'])
tabela = tabela[['5', '1', '4', '0']]

In [21]:
tabela = tabela.rename(columns={'5':'Casos confirmados', '1':'Casos descartados', '4':'Casos suspeitos', '0':'Não informados'})

In [22]:
tabela['Total testes'] = tabela.sum(1)
tabela['Positividade (%)'] = tabela['Casos confirmados'].div(tabela['Total testes'])

In [24]:
tabela.to_excel('output/TABELA_testes_sivep.xlsx')

In [25]:
tabela

CLASSI_FIN,Casos confirmados,Casos descartados,Casos suspeitos,Não informados,Total testes,Positividade (%)
SE,,,,,,
202001,1.0,NaN,8.0,NaN,9.0,0.111111
202002,1.0,NaN,27.0,2.0,30.0,0.033333
202003,1.0,NaN,26.0,NaN,27.0,0.037037
202004,NaN,NaN,23.0,NaN,23.0,NaN
202005,NaN,17.0,32.0,NaN,49.0,NaN
...,...,...,...,...,...,...
202113,2306.0,NaN,319.0,830.0,3455.0,0.667438
202114,1904.0,19.0,305.0,914.0,3142.0,0.605983
202115,1560.0,NaN,230.0,1216.0,3006.0,0.518962
